# Spam Filtering Using The [Enron Dataset][1]
[1]: [http://www.aueb.gr/users/ion/data/enron-spam/]

In [1]:
from pymldb import Connection
mldb = Connection('http://localhost/')

Let's start by loading the dataset. We have already merged the different email files in a sensible manner into a .csv file, which we've made available online. Since this dataset is actually made up of six different datasets, we'll restrict ourself to the first one for simplicity, using a "where" clause.

In [2]:
print mldb.post('/v1/procedures', {
    'type': 'import.text',
    'params': {
        'dataFileUrl': 'http://public.mldb.ai/datasets/enron.csv.gz',
        'outputDataset': 'enron_data',
        'named': "'enron_' + dataset + '_mail_' + index",
        'where': 'dataset = 1',
        'runOnCreation': True
        }
    })

<Response [201]>


This is what the dataset looks like.

*index*: order in which the emails arrived in the user's inbox  
*msg*: actual content of the email  
*label*: was the email legitimate (*ham*) or not (*spam*)  

In [3]:
mldb.query('select index, msg, label from enron_data order by index limit 10')

,index,label,msg
_rowName,,,
enron_1_mail_0,0,spam,Subject: dobmeos with hgh my energy level has ...
enron_1_mail_1,1,spam,Subject: your prescription is ready . . oxwq s...
enron_1_mail_2,2,ham,Subject: christmas tree farm pictures
enron_1_mail_3,3,ham,"Subject: vastar resources , inc .gary , produc..."
enron_1_mail_4,4,ham,Subject: calpine daily gas nomination- calpine...
enron_1_mail_5,5,ham,Subject: re : issuefyi - see note below - alre...
enron_1_mail_6,6,ham,Subject: meter 7268 nov allocationfyi .- - - -...
enron_1_mail_7,7,spam,Subject: get that new car 8434people nowthe we...
enron_1_mail_8,8,ham,"Subject: mcmullen gas for 11 / 99jackie ,since..."


Let's create a *sql.expression* that will simply tokenize the emails into a bag of words. Those will be our features on which we will train a classifier.

In [4]:
print mldb.put('/v1/functions/bow', {
    'type': 'sql.expression',
    'params': {
        'expression': """
            tokenize(msg, {splitchars: ' \n', quotechar: ''}) as bow
            """
    }
})

<Response [201]>


Then we can generate the features for the whole dataset, and write them into a new dataset, using the *transform* procedure.

In [5]:
print mldb.put('/v1/procedures/generate_feats', {
    'type': 'transform',
    'params': {
        'inputData': """
            select bow({msg}) as features, label = 'spam' as label
            from enron_data
            """,
        'outputDataset': 'enron_features',
        'runOnCreation': True
    }
})

<Response [201]>


Here is a snapshot of the sparse feature matrix:

In [6]:
mldb.query('select * from enron_features limit 10')

,features.bow.&,features.bow.-,"features.bow."".al""","features.bow."".last""","features.bow."".""",features.bow./,features.bow.00daren,features.bow.00,features.bow.1,features.bow.273,...,features.bow.transactions,features.bow.transportation,features.bow.undermine,features.bow.under,features.bow.use,features.bow.using,features.bow.wants,features.bow.weren,features.bow.were,features.bow.whose
_rowName,,,,,,,,,,,,,,,,,,,,,
enron_1_mail_3323,1,2,1,1,5,3,1,2,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3284,NaN,NaN,NaN,NaN,4,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_1291,NaN,48,NaN,NaN,2,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_1304,NaN,50,NaN,NaN,4,8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3040,NaN,6,NaN,NaN,10,2,NaN,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_2372,NaN,9,NaN,NaN,11,2,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_2881,NaN,NaN,NaN,NaN,2,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_2706,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_2317,NaN,49,NaN,NaN,1,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's train a very simple classifier, by training on the first half of the messages, and testing on the second half. This classifier will give a score to every email, and we can then choose a threshold where everything above the threshold is classified as spam, and every thing below as ham.

In [7]:
n = mldb.get('/v1/query', q='select count(*) as n from enron_features',
             format='aos').json()[0]['n']

res = mldb.put('/v1/procedures/experiment', {
    'type': 'classifier.experiment',
    'params': {
        'experimentName': 'enron_experiment1',
        'inputData': 'select {features.*} as features, label from enron_features',
        # for now 50/50 split in time, but we might do something more
        # fancy in a later version!
        'datasetFolds': [{
           'trainingLimit': n // 2,
           'testingOffset': n // 2,
           'trainingOrderBy': 'index',
           'testingOrderBy': 'index'
        }],
        'modelFileUrlPattern': 'file://enron_model_$runid.cls',
        'algorithm': 'bbdt',
        'runOnCreation': True
    }
})
print res

<Response [201]>


In [8]:
print 'AUC =', res.json()['status']['firstRun']['status']['aggregatedTest']['auc']['mean']

AUC = 0.99452149827


Not a bad AUC for a model that simple. But [the AUC score of a classifier is only a very generic measure of performance][1]. When having a specific problem like spam filtering, we're better off using a performance metric that truly matches our intuition about what a good spam filter ought to be. Namely, a good spam filtering algorithm should almost never flag as spam a legitime email, while keeping your inbox as spam-free as possible. This is what should be used to choose the threshold for the classifier, and then to measure its performance.

So instead of the AUC (that doesn't pick a specific threshold but uses all of them), let's use as our performance metric the best [$F_{0.05}$ score][2], which gives 20 times more importance to precision than recall. In other words, this metric represents the fact that classifying as spam **only** what is really spam is 20 times more important than finding all the spam.

Let's see how we are doing with that metric.
[1]: http://mldb.ai/blog/posts/2016/01/ml-meets-economics/
[2]: https://en.wikipedia.org/wiki/F1_score

In [9]:
print mldb.put('/v1/functions/enron_score', {
    'type': 'sql.expression',
    'params': {
        'expression': """
            (1 + pow(ratio, 2)) * (precision * recall) / (precision * pow(ratio, 2) + recall) as enron_score
            """
    }
})

<Response [201]>


In [10]:
mldb.query("""
    select "truePositives", "trueNegatives", "falsePositives", "falseNegatives", precision, recall, score,
           enron_score({precision, recall, ratio:0.05}) as *
    from enron_experiment1_results_0
    order by enron_score desc
""")

,enron_score,falseNegatives,falsePositives,precision,recall,score,trueNegatives,truePositives
_rowName,,,,,,,,
enron_1_mail_1166,0.998106,328,0,1.000000,0.567852,2.148124,1827,431
enron_1_mail_979,0.998096,329,0,1.000000,0.566535,2.150652,1827,430
enron_1_mail_1163,0.998085,330,0,1.000000,0.565217,2.155546,1827,429
enron_1_mail_1825,0.998075,331,0,1.000000,0.563900,2.166088,1827,428
enron_1_mail_943,0.998065,332,0,1.000000,0.562582,2.166368,1827,427
enron_1_mail_2364,0.998054,333,0,1.000000,0.561265,2.171287,1827,426
enron_1_mail_3120,0.998044,334,0,1.000000,0.559947,2.183409,1827,425
enron_1_mail_363,0.998034,335,0,1.000000,0.558630,2.184853,1827,424
enron_1_mail_3760,0.998023,336,0,1.000000,0.557312,2.188323,1827,423


    As you can see, the best threshold is one where in case of doubt, almost everything is classified as "ham". This leads to 439 spam messages in the inbox, (only) 1 ham wrongly filtered as spam and 2146 well classified emails. Now how can we improve this?

# To Be Continued...